Predict labels for images

Use all pixel information as input

Convolution patch

Stride 1 x 1 (how we move)

Example (1 x 1 stride)

3 x 3 x 1 image
2 x 2 x 1 filter

Output is 2 x 2

Input
```
[
    1, 2, 3,
    4, 5, 6,
    7, 8, 9
]
```
Filter
```
[
    0.1, 0.5,
    0.3, 0.4
]
```
Output
```
[
       4.3, ...
]
```

$ w^Tx + b $ (bias is not always used)

Don't forget about 0 padding!

Filter is almost used as a feature map

The result of the filter is an activation map

Can build layers of activation maps by running multiple filters and stacking the results ontop of each other 2 x 2 matrix to create 3d matrix

Apply the filter, then threshold with RELU each number in the matrix

### Max Pooling

Move by filter size as stride, do an argmax / average, to get the resultant value of the filter applied

### Locally Connected Features

#### Fully connected neural network

Each pixel goes to each neuron !

#### Locally connected neural network

Parts of an image to go a neuron (pixels get combined into a singlar set of information, most valuable information that is)

Smaller weights / more flexible

In [96]:
import torch as t
input = t.randn(3, 16, 50)
input

tensor([[[ 0.2890,  0.6837,  0.0262,  ..., -0.0653,  0.8718,  0.6821],
         [ 0.6440,  0.0763, -0.6445,  ..., -0.1091,  0.2596, -0.9315],
         [-0.5382,  0.8802, -0.3386,  ..., -1.2852, -0.9651, -1.2392],
         ...,
         [ 0.8056,  0.0134, -1.8466,  ..., -0.5810, -0.6743, -1.1742],
         [-0.3886,  0.2824,  0.0743,  ..., -0.3317,  0.7260,  0.2819],
         [-0.7559, -0.2986,  0.4459,  ..., -0.2426, -0.5779, -1.7032]],

        [[-0.0205, -3.2954,  0.5483,  ...,  1.1553, -1.3097, -1.6172],
         [ 0.2934, -0.8175, -1.4015,  ...,  0.0356,  1.1573,  1.0726],
         [-0.2480,  1.2908,  0.3898,  ..., -0.4925, -0.3988,  1.6352],
         ...,
         [ 0.5866,  0.7395,  1.5658,  ..., -0.9274, -2.0744,  0.8876],
         [ 0.7643,  0.4222, -0.0477,  ..., -1.6724,  0.6320,  0.5566],
         [ 0.7445,  0.9054,  0.3023,  ...,  0.3868,  0.3438, -2.1708]],

        [[-0.8475,  0.0121,  1.8199,  ..., -0.0637,  2.1401,  0.7824],
         [ 0.2190,  1.5356,  0.8098,  ..., -0

In [97]:
m = t.nn.Conv2d(3, 1, 3) # the 3x output channel filters are random, 
# this says colors (channels) (rows)/(height) (columns) / (width)
m
# https://discuss.pytorch.org/t/explaination-of-conv2d/8082

Conv2d(3, 1, kernel_size=(3, 3), stride=(1, 1))

In [99]:
output = m(input.view(1, 3, 16, 50))

In [72]:
from torchvision import datasets, transforms

# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = t.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = t.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [92]:
#kk = t.nn.Conv2d(1, 10, kernel_size=5)
#kk(train_dataset[0][0].view(1,1,28,28))
#kk(train_dataset[0][0].view(1,1,28,28))[0][1]

In [107]:
import torch.nn.functional as F

class Net(t.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv_1 = t.nn.Conv2d(1, 10, kernel_size=5)
        self.conv_2 = t.nn.Conv2d(10, 20, kernel_size=5)
        self.mp = t.nn.MaxPool2d(2)
        self.fc = t.nn.Linear(320, 10)
    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv_1(x)))
        x = F.relu(self.mp(self.conv_2(x)))
        x = x.view(in_size, -1) # flatten 1d
        x = self.fc(x)
        return F.log_softmax(x)

In [108]:
model = Net()
criterion = t.nn.CrossEntropyLoss()
optimizer = t.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        output = model.forward(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    model.eval() # put in eval mode, no gradients
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data[0]
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 10):
    train(epoch)
    test()

/Users/achang/miniconda2/envs/deep-learning-lab/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()
/Users/achang/miniconda2/envs/deep-learning-lab/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  app.launch_new_instance()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303888
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.302836
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.285493
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.262688
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.249491
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.222077
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.203959
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.155921
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.105059
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.912824
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.753867
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.655697
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.433314
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.117218
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.824135
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.037394
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.956977
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.810087
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.798395
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.663074
Train Epoch: 1 [12800/60000 (

/Users/achang/miniconda2/envs/deep-learning-lab/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



Test set: Average loss: 0.0033, Accuracy: 9336/10000 (93%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.286455
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.131291
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.342729
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.106738
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.211723
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.151955
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.483460
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.179391
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.149503
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.186172
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.107293
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.271574
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.109717
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.139081
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.201938
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.119543
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.311298
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.120559
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.171744
Train Epoch: 2 [12

Train Epoch: 3 [44160/60000 (74%)]	Loss: 0.106641
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.056674
Train Epoch: 3 [45440/60000 (76%)]	Loss: 0.041254
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.071185
Train Epoch: 3 [46720/60000 (78%)]	Loss: 0.048113
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.136970
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.075927
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.127025
Train Epoch: 3 [49280/60000 (82%)]	Loss: 0.102025
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.077010
Train Epoch: 3 [50560/60000 (84%)]	Loss: 0.225173
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.049163
Train Epoch: 3 [51840/60000 (86%)]	Loss: 0.145063
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.108748
Train Epoch: 3 [53120/60000 (88%)]	Loss: 0.028504
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.047202
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.099304
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.046519
Train Epoch: 3 [55680/60000 (93%)]	Loss: 0.121998
Train Epoch: 3 [56320/60000 (94%)]	Loss: 0.073549


Train Epoch: 5 [28160/60000 (47%)]	Loss: 0.023148
Train Epoch: 5 [28800/60000 (48%)]	Loss: 0.042108
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.076238
Train Epoch: 5 [30080/60000 (50%)]	Loss: 0.080269
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.065941
Train Epoch: 5 [31360/60000 (52%)]	Loss: 0.092719
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.025962
Train Epoch: 5 [32640/60000 (54%)]	Loss: 0.045163
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.032578
Train Epoch: 5 [33920/60000 (57%)]	Loss: 0.058909
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.065594
Train Epoch: 5 [35200/60000 (59%)]	Loss: 0.021723
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.075855
Train Epoch: 5 [36480/60000 (61%)]	Loss: 0.116822
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.049561
Train Epoch: 5 [37760/60000 (63%)]	Loss: 0.009007
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.071910
Train Epoch: 5 [39040/60000 (65%)]	Loss: 0.071132
Train Epoch: 5 [39680/60000 (66%)]	Loss: 0.090647
Train Epoch: 5 [40320/60000 (67%)]	Loss: 0.024161


Train Epoch: 7 [12160/60000 (20%)]	Loss: 0.070847
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.019555
Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.075677
Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.091412
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.032312
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.027793
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.061876
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.027730
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.047979
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.105535
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.017476
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.039839
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.085911
Train Epoch: 7 [20480/60000 (34%)]	Loss: 0.023908
Train Epoch: 7 [21120/60000 (35%)]	Loss: 0.022341
Train Epoch: 7 [21760/60000 (36%)]	Loss: 0.151817
Train Epoch: 7 [22400/60000 (37%)]	Loss: 0.031445
Train Epoch: 7 [23040/60000 (38%)]	Loss: 0.018184
Train Epoch: 7 [23680/60000 (39%)]	Loss: 0.058539
Train Epoch: 7 [24320/60000 (41%)]	Loss: 0.038582


Train Epoch: 8 [56960/60000 (95%)]	Loss: 0.013571
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.070616
Train Epoch: 8 [58240/60000 (97%)]	Loss: 0.046204
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.123058
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.111738

Test set: Average loss: 0.0008, Accuracy: 9826/10000 (98%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.013790
Train Epoch: 9 [640/60000 (1%)]	Loss: 0.042096
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.135682
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.065757
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.095121
Train Epoch: 9 [3200/60000 (5%)]	Loss: 0.011276
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.120222
Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.060439
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.036583
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.048820
Train Epoch: 9 [6400/60000 (11%)]	Loss: 0.106233
Train Epoch: 9 [7040/60000 (12%)]	Loss: 0.116785
Train Epoch: 9 [7680/60000 (13%)]	Loss: 0.023472
Train Epoch: 9 [8320/60000 (14%)]	Loss: 0.103392
Train Epoch: 9 [